## 1. Import thư viện

In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.preprocessing import StandardScaler
import sklearn.metrics as metric
from sklearn.metrics import make_scorer, accuracy_score, f1_score, precision_score, recall_score
import xgboost as xgb
from xgboost import plot_importance
import matplotlib.pyplot as plt
import random

## 2. Load dữ liệu<br>
(status_per_min_01 là dữ liệu được fillNaN 0)

In [2]:
clean_data_file_name = r'..\preprocessing\clean_data\status_per_min_01.csv'

In [3]:
df = pd.read_csv(clean_data_file_name)
new_df = df.copy()

## 3. Chuẩn hóa dữ liệu

3.1. Chuẩn hóa cột Chênh lệch vàng, tỷ lệ thắng của người chơi

In [4]:
win_rate_columns = df.columns[:20]
gold_columns = df.columns[471:512]
new_df[win_rate_columns] = StandardScaler().fit_transform(df[win_rate_columns])
new_df[gold_columns] = StandardScaler().fit_transform(df[gold_columns])
new_df = pd.DataFrame(new_df, columns=df.columns)

3.2. Xóa các cột từ phút thứ 0 đến phút thứ 5 vì không mang nhiều ý nghĩa

In [5]:
level_columns_drop = df.columns[20:50]
gold_columns_drop = df.columns[471:477]
turrets_columns_drop = df.columns[225:273]
monstermonster_columns_drop = df.columns[512:562]
columns_to_drop = list(level_columns_drop) + list(gold_columns_drop) + list(turrets_columns_drop) + list(monstermonster_columns_drop)
new_df = new_df.drop(columns=columns_to_drop)
new_df['label'] = new_df['label'].replace(-1, 0)

3.3. Xóa các cột từ phút thứ 36 đến 40 vì có ít trận kéo dài đến thời điểm đó

In [6]:
columns_to_drop_01 = [col for col in df.columns if col[-2:] in {'36', '37', '38', '39', '40'}]
new_df = new_df.drop(columns=columns_to_drop_01)

3.4. Phân chia dữ liệu, 1 phần dùng để huấn luyện, 1 phần coi như dữ liệu mới hoàn toàn

In [7]:
old_df, new_df = train_test_split(new_df, test_size=0.3, random_state=1)

In [8]:
old_X = old_df.drop(columns=['label'])
old_Y = old_df['label']

3.5. Chia dữ liệu huấn luyện theo tỷ lệ 7-3

In [9]:
old_X_training, old_X_test, old_Y_training, old_Y_test = train_test_split(old_X, old_Y, test_size=0.3, random_state=0)

3.6. Khai báo model với hyperparameter được chọn bằng cách sử dụng 5-fold validation

In [10]:
XG_boost = xgb.XGBClassifier(n_estimators = 50, max_depth = 11, learning_rate = 0.3).fit(old_X_training, old_Y_training)

In [11]:
# param_grid = {
#     'n_estimators': [30, 40, 50],
#     'learning_rate': [0.2, 0.3, 0.4],
#     'max_depth': [7, 9, 11],
# }

# grid_search = GridSearchCV(
#     estimator=xgb.XGBClassifier(random_state=42),
#     param_grid=param_grid,
#     cv=5,
#     scoring='accuracy',
#     verbose=1
# ).fit(old_X_training, old_Y_training)
# XG_boost = grid_search.best_estimator_

In [12]:
# grid_search.best_params_

## 4.Performance trên old_df

In [13]:
# Accuracy
print('Train')
print(metric.accuracy_score(old_Y_training, XG_boost.predict(old_X_training)))
print('Test')
print(metric.accuracy_score(old_Y_test, XG_boost.predict(old_X_test)))

Train
1.0
Test
0.9525709219858156


In [14]:
# F1 score
print('Train')
print(metric.f1_score(old_Y_training, XG_boost.predict(old_X_training), pos_label=1))
print('Test')
print(metric.f1_score(old_Y_test, XG_boost.predict(old_X_test), pos_label=1))

Train
1.0
Test
0.9530701754385965


In [15]:
# Precision Score
print('Train')
print(metric.precision_score(old_Y_training, XG_boost.predict(old_X_training), pos_label=1))
print('Test')
print(metric.precision_score(old_Y_test, XG_boost.predict(old_X_test), pos_label=1))

Train
1.0
Test
0.9589585172109444


## 5. Performance trên new_dfdf

5.1. Tạo phút x ngẫu nhiên cho mỗi trận. Ví dụ x = 15, các giá trị từ phút thứ 0 đến 15 sẽ giữ nguyên, các giá trị từ phút thứ 15 đến 15 sẽ bằng 0

In [16]:
random_matches = new_df.copy()
num_matches = new_df.shape[0]

random_minutes = [random.randint(5, 36) for _ in range(num_matches)]
random_matches['minute'] = random_minutes

def get_data_until_minute(row):
    minute = row['minute']
    selected_columns = list(random_matches.columns[:20])
    for col in new_df.columns:
        last_part = col.split('_')[-1]
        if last_part.isdigit() and int(last_part) <= minute:
            selected_columns.append(col)
    selected_columns.append('label')
    return row[selected_columns]

filtered_data = random_matches.apply(get_data_until_minute, axis=1)
filtered_data = filtered_data.reindex(columns=random_matches.columns)

X_real_time = filtered_data.drop(columns=["label", "minute"])
Y_real_time = filtered_data["label"]

X_real_time = X_real_time.fillna(0)  
Y_real_time = Y_real_time.fillna(0) 

5.2. Performance sử dụng cross validation

In [17]:
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'f1': make_scorer(f1_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score)
}

results = cross_validate(
    estimator=XG_boost, 
    X=X_real_time, 
    y=Y_real_time, 
    scoring=scoring, 
    cv=10
)

In [18]:
# Giá trị trung bình của các metric
print('Test Accuracy:', results['test_accuracy'].mean())

print('Test Precision:', results['test_precision'].mean())

print('Test Recall:', results['test_recall'].mean())

print('Test F1 Score:', results['test_f1'].mean())

Test Accuracy: 0.7635420578747172
Test Precision: 0.764439513149919
Test Recall: 0.7520663452572267
Test F1 Score: 0.7579066283340137


5.3. Dự đoán trên new_df

In [19]:
Y_predict = XG_boost.predict(X_real_time)
predicted_proba = XG_boost.predict_proba(X_real_time)
# Thêm các cột xác suất chọn label
random_matches['per_red_win'] = predicted_proba[:, 0].round(2) 
random_matches['per_blue_win'] = predicted_proba[:, 1].round(2) 
random_matches['predicted_label'] = Y_predict

In [20]:
result_predict = random_matches[['per_blue_win', 'per_red_win', 'minute', 'predicted_label', 'label']]

Số trận có x < 10 và bị dự đoán sai

In [21]:
result_predict[(result_predict['minute'] < 10) & (result_predict['predicted_label'] != result_predict['label'])].shape[0]

522

Số trận có x < 20 và bị dự đoán sai

In [22]:
result_predict[(result_predict['minute'] < 20) & (result_predict['predicted_label'] != result_predict['label'])].shape[0]

1500

Số trận có x < 30 và bị dự đoán sai

In [23]:
result_predict[(result_predict['minute'] < 30) & (result_predict['predicted_label'] != result_predict['label'])].shape[0]

1990

Số trận có x <== 35 và bị dự đoán sai

In [24]:
result_predict[(result_predict['minute'] < 36) & (result_predict['predicted_label'] != result_predict['label'])].shape[0]

2051

In [25]:
result_predict.to_csv(r"D:\Learning\ML\BTL\analyzing\result.csv")